## Notebook criado apenas para tests. A aplicação não depende deste arquivo

In [5]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import plotly.express as px

credentials = service_account.Credentials.from_service_account_file("cea-adw-johann.json")

client = bigquery.Client(credentials=credentials)

In [2]:
# Perform a query.
DIMENSIONS = ['nome_produto', 'tipo_cartao']

for dimension in DIMENSIONS:
    QUERY = (
        f'''
        SELECT COUNT(*) QNTD, {dimension} 
        FROM `dbt_adventure_works.vendas` 
        GROUP BY `{dimension}`
        ORDER BY QNTD DESC
        ''')
    pedidos = client.query(QUERY).to_dataframe()  # API request

    fig = px.bar(pedidos[:15], x='QNTD', y=dimension, orientation='h')
    fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
    fig.show()

In [3]:
import sys
sys.path.append(r"C:\Users\johann.figueiredo\DEV\academy-dbt-johann\app")
from utils import total_negociado_por_dimensao

quantidade_comprada = client.query(total_negociado_por_dimensao('nome_produto')).to_dataframe()

fig = px.bar(quantidade_comprada[:15], x='total', y='nome_produto', orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show()

In [4]:
dimension = 'ano_pedido'
valor = 2014

QUERY = (
    f'''
        SELECT ROUND(AVG(preco_unitario), 2) ticket_medio, nome_produto, ano_pedido, mes_pedido
        FROM `dbt_adventure_works.vendas`
        WHERE {dimension} = {valor}
        GROUP BY nome_produto, ano_pedido, mes_pedido
        ORDER BY ano_pedido DESC, mes_pedido
    ''')
pedidos = client.query(QUERY).to_dataframe()  # API request

fig = px.bar(pedidos[:15], x='ticket_medio', y='nome_produto', orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show()

In [5]:
#serie temporal

dimension = 'ano_pedido, mes_pedido'

QUERY = (
    f'''
    WITH pedidos_unicos as (
    SELECT DISTINCT pedido_id, total_pedido, ano_pedido, mes_pedido
    FROM `dbt_adventure_works.vendas` 
    )

    SELECT SUM(total_pedido) total, (mes_pedido || '/' || ano_pedido) as date
    FROM pedidos_unicos
    GROUP BY ano_pedido, mes_pedido
    ORDER BY ano_pedido, mes_pedido
    ''')
pedidos = client.query(QUERY).to_dataframe()  # API request

fig = px.area(pedidos, x='date', y='total')
fig.update_layout({'xaxis': {'nticks': 10, 'tick0': 100}})
fig.show()

In [9]:
dimension = 'sigla_3'

QUERY = (
    f'''
    SELECT COUNT(*) QNTD, {dimension} pais
    FROM `dbt_adventure_works.vendas` 
    GROUP BY `{dimension}`
    ORDER BY QNTD DESC
    ''')
pedidos = client.query(QUERY).to_dataframe()  # API request

fig = px.choropleth(pedidos, locations='pais', locationmode='ISO-3', color='QNTD')
fig.show()

In [15]:
# melhores clientes

QUERY = f'''
    WITH pedidos_unicos as (
        SELECT DISTINCT pedido_id, total_pedido, nome_loja
        FROM `cea-adw-johann.dbt_adventure_works.vendas`
    )

    SELECT ROUND(SUM(total_pedido)) total, nome_loja
    FROM pedidos_unicos
    GROUP BY nome_loja
'''


clientes = client.query(QUERY).to_dataframe() 

fig = px.bar(clientes[:10], x='total', y='nome_loja', orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show()



QUERY = f'''
    WITH pedidos_unicos as (
        SELECT DISTINCT pedido_id, total_pedido, data_pedido, nome_cliente
        FROM `cea-adw-johann.dbt_adventure_works.vendas`
        WHERE nome_loja = 'pessoa_fisica'
    )

    SELECT ROUND(SUM(total_pedido)) total, nome_cliente
    FROM pedidos_unicos
    GROUP BY nome_cliente
'''


clientes = client.query(QUERY).to_dataframe() 

fig = px.bar(clientes[:10], x='total', y='nome_cliente', orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show()